# Predict

In [15]:
import sys
sys.path.insert(0,"/home/gridsan/hwpang/Software/RMG-Py/")
sys.path.insert(0,"..")

import json
import pandas as pd

from rmgpy.data.thermo import ThermoDatabase, ThermoGroups

from tree.thermo import ThermoGroups as SIDTThermoGroups
from tree.utils import make_mol
from tree.parameters import Ts

# Load test data

In [16]:
hbi_unc_df = pd.read_csv("../data/hbi_unc.csv")

In [17]:
split = "cluster"

with open(f"../data/splits/{split}.json", "r") as f:
    train_inds, test_inds = json.load(f)

In [18]:
test_df = hbi_unc_df.loc[test_inds, :]
test_df

,radical_smiles,resonance_radical_smiles,resonance_radical_num_rotatable_bonds,radical_H298 (kcal/mol),radical_Sint298 (cal/mol/K),radical_source,radical_level_of_theory,radical_Cp300 (cal/mol/K),radical_Cp400 (cal/mol/K),radical_Cp500 (cal/mol/K),...,unc_closed_shell_Cp1500 (cal/mol/K),unc_HBI_H298 (kcal/mol),unc_HBI_Sint298 (cal/mol/K),unc_HBI_Cp300 (cal/mol/K),unc_HBI_Cp400 (cal/mol/K),unc_HBI_Cp500 (cal/mol/K),unc_HBI_Cp600 (cal/mol/K),unc_HBI_Cp800 (cal/mol/K),unc_HBI_Cp1000 (cal/mol/K),unc_HBI_Cp1500 (cal/mol/K)
352,C[C](C)C(C)(C)OOC(C)(C)C(C)=O,C[C](C)C(C)(C)OOC(C)(C)C(C)=O,12,-82.664553,166.111415,dong_pio_liang.py,CBS-QB3,73.330611,88.694774,102.300740,...,2.0,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
456,C[C](C)C(C)(C)OOC(C)=O,C[C](C)C(C)(C)OOC(C)=O,8,-72.908785,141.625884,dong_pio_liang.py,CBS-QB3,52.121340,63.668323,74.176599,...,2.0,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
555,C[C](C)C(C)(C)OOC(C)(C)C(C)C,C[C](C)C(C)(C)OOC(C)(C)C(C)C,13,-65.900457,160.026007,dong_pio_liang.py,CBS-QB3,77.841301,95.201630,110.339355,...,2.0,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
592,C[C](C)C(C)(C)OOCC(C)=O,C[C](C)C(C)(C)OOCC(C)=O,10,-62.515449,152.049209,dong_pio_liang.py,CBS-QB3,61.751038,73.543577,84.298041,...,2.0,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
630,COOC(C)(C)C(C)(C)O[O],COOC(C)(C)C(C)(C)O[O],9,-59.487249,131.604726,dong_pio_liang.py,CBS-QB3,57.280379,68.179164,77.644730,...,2.0,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2706,[CH2]C=CC1C=CC1,[CH2]C=CC1C=CC1,2,83.910003,89.773882,dong_pio_liang.py,CBS-QB3,31.089321,38.791510,45.477610,...,2.0,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2707,[CH2]C=CC1C=CC1,C=C[CH]C1C=CC1,2,83.910003,89.773882,dong_pio_liang.py,CBS-QB3,31.089321,38.791510,45.477610,...,2.0,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2726,C1=CC2[C](C1)C1C=CC2C1,C1=CC2[C](C1)C1C=CC2C1,0,88.928510,86.909073,pang.py,CBS-QB3,33.752708,46.798126,57.999145,...,1.0,2.4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2766,[CH]=C1C=CC=C1,[CH]=C1C=CC=C1,0,112.283864,73.905617,dong_pio_liang.py,CBS-QB3,21.129746,27.530091,32.691325,...,2.0,5.2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [19]:
mols = test_df["resonance_radical_smiles"].apply(make_mol)
mols = mols.to_list()

# Load thermo database

In [20]:
def make_prediction(thermo_database, tree, mol):
    atoms = {"*": atom for atom in mol.atoms if atom.radical_electrons==1}
    return thermo_database._add_group_thermo_data(None, tree, mol, atoms)[0]

In [21]:
thermo_database = ThermoDatabase()

# Predict with tree

In [25]:
run_dir = f"split-{split}_run-retrain_empirical"
tree = SIDTThermoGroups().load(f"../models/{run_dir}/tree.py", thermo_database.local_context, thermo_database.global_context)

In [26]:
thermos = [make_prediction(thermo_database, sidt_tree, mol) for mol in mols]

In [27]:
test_result_df = test_df[["resonance_radical_smiles"]]
test_result_df["HBI_H298 (kcal/mol)"] = [thermo.H298.value_si/4184 for thermo in thermos]
test_result_df["unc_HBI_H298 (kcal/mol)"] = [thermo.H298.uncertainty_si/4184 for thermo in thermos]
test_result_df["HBI_Sint298 (cal/mol/K)"] = [thermo.S298.value_si/4.184 for thermo in thermos]
test_result_df["unc_HBI_Sint298 (cal/mol/K)"] = [thermo.S298.uncertainty_si/4.184 for thermo in thermos]
for i, T in enumerate(Ts):
    test_result_df[f"HBI_Cp{T} (cal/mol/K)"] = [thermo.Cpdata.value_si[i]/4.184 for thermo in thermos]
    test_result_df[f"unc_HBI_Cp{T} (cal/mol/K)"] = [thermo.Cpdata.uncertainty_si[i]/4.184 for thermo in thermos]
test_result_df["comment"] = [thermo.comment for thermo in thermos]

/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/ipykernel_launcher.

In [28]:

test_result_df.to_csv(f"../models/{run_dir}/test.csv", index=False)